In [1]:
import sympy as sp

# Variable dummy Nc (numero de componentes)
nc = sp.symbols("Nc", integer=True)

# Indices
# Para derivadas de n
i = sp.symbols("i", cls=sp.Idx)
j = sp.symbols("j", cls=sp.Idx)

# Indices para variables
k = sp.symbols("k", cls=sp.Idx)
l = sp.symbols("l", cls=sp.Idx)

# Variables
n = sp.IndexedBase("n", shape=(nc,))
T = sp.symbols("T")
V = sp.symbols("V")


r = sp.IndexedBase('r')
q = sp.IndexedBase('q')

a = sp.IndexedBase('a')
b = sp.IndexedBase('b')

# Definimos φ_k como función general
phi = sp.Function('phi')

# Expresión: φ_k = phi(n[k], T, P)
phi_k = phi(n[k])

In [175]:
# Subclase de Function
class CFunction(sp.Function):
    @classmethod
    def eval(cls, n_k, T):
        # No simplificamos nada en eval
        pass
       
    def __new__(cls, *args, name=None, display_name=None, diff_list=[], is_diff=False, **kwargs):
        obj = super().__new__(cls, *args, **kwargs)
      
        if name:
            if not is_diff:
                obj._custom_name = f"{name}_" + args[0].args[1].__str__()
            else:
                obj._custom_name = f"{name}"
        else:
            obj._custom_name = cls.__name__
    
        if not display_name:
            obj._display_name = obj._custom_name
        else:
            obj._display_name = display_name
            
        obj._diff_list = diff_list

        return obj

    def __str__(self):
        return f"{self._display_name}({', '.join(map(str, self.args))})"

    def __repr__(self):
        return f"{self._display_name}({', '.join(map(str, self.args))})"
    
    def _sympystr(self, printer):
        return f"{self._display_name}"

    def _latex(self, printer):
        return f"{self._display_name}"
    
    def diff(self, sym, **kwargs):
        # Solo tratamos derivadas respecto a elementos de n[i]
        if isinstance(sym, sp.Indexed) and sym.base == n:
            kdx = self.args[0].args[0]  # Extrae el índice k de n[k]
            idx = sym.args[1]           # Extrae el índice i de n[i]
            
            new_dx =f"\\partial n_{idx}"
        else:
            new_dx = f"\\partial {sym.__str__()}"
            
        diff_list = self._diff_list.copy()
        
        if new_dx not in diff_list:
            diff_list.append(new_dx)
        
        full_dx = ""

        for dx in diff_list:
            full_dx += f"{dx}"
        
        symbol_string = "\\frac{" + f"\\partial {self._custom_name}" + "}{" + full_dx + "}"
                    
        return CFunction(*self.args[:], name=self._custom_name, display_name=symbol_string, is_diff=True, diff_list=diff_list, **kwargs)

In [180]:
# Funcion de n[k] y T
phi_k = CFunction(n[k], T, name=r'\phi')
phi_k

\phi_k(n[k], T)

In [181]:
sp.diff(phi_k, n[i])

\frac{\partial \phi_k}{\partial n_i}(n[k], T)

In [183]:
exp = sp.Sum(phi_k * n[k], (i, 1, nc))
exp

Sum(\frac{\partial \phi_k}{\partial n_i}*n[k], (i, 1, Nc))

In [184]:
sp.diff(exp, n[i])

Sum(\frac{\partial \phi_k}{\partial n_i}*KroneckerDelta(k, i) + \frac{\partial \phi_k}{\partial n_i}*n[k], (i, 1, Nc))

In [61]:
# OLD
class CFunction(sp.Function):
    @classmethod
    def eval(cls, n_k, T):
        # No simplificamos nada en eval
        pass

    def fdiff(self, argindex=1):
        raise NotImplementedError("Use .diff(var) for partial derivatives.")

    def diff(self, sym, **kwargs):
        # Solo tratamos derivadas respecto a elementos de n[i]
        if isinstance(sym, sp.Indexed) and sym.base == n:
            k = self.args[0].args[0]  # Extrae el índice k de n[k]
            i = sym.args[0]           # Extrae el índice i de n[i]
                       
            return CFunction(*self.args[:], name=r'\frac{\partial \phi}{\partial n}')
        else:
            # Por defecto: usa el método base
            return super().diff(sym, **kwargs)
        
    def __new__(cls, *args, name=None, **kwargs):
        obj = super().__new__(cls, *args, **kwargs)
        obj._custom_name = name or cls.__name__
        return obj

    def __str__(self):
        return f"{self._custom_name}({', '.join(map(str, self.args))})"

    __repr__ = __str__
    
    def _sympystr(self, printer):
        return f"{self._custom_name}({', '.join(printer._print(arg) for arg in self.args)})"

    def _latex(self, printer):
        return f"{self._custom_name}\\left({', '.join(printer._print(arg) for arg in self.args)}\\right)"


k = sp.Symbol('k', integer=True)
phi_k = CFunction(n[k], T, name=r'\phi')

phi_k

\phi(n[k], T)

In [22]:
def simple_kronecker(expr):
    solved = sp.diff(expr, n[i])
    
    solved = solved.doit().subs({i >= 1: True, nc >= i: True})

    return solved